#**Studienleistung 2 IL_2 Sascha Lingl und Paul Himstedt**

## Coin Caller

### Formulierung der Frage bezüglich aktuellen Geschehnissen am Krypto Markt


In [1]:
question_input_string = input("Your Question: ")
person_input_string = input("person, empty if irrelevant: ")
print(question_input_string,person_input_string)

Your Question: Bitcoin performance
person, empty if irrelevant: 
Bitcoin performance 


### Hier werden die nötigen Installationen und Imports vorgenommen

In [2]:
#First Installation can take up to 5 min
# Install the latest release of Haystack in your own environment 
! pip install farm-haystack

# Install the latest master of Haystack
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4

#imports
import pandas as pd
import spacy
import tweepy as tw
import os
import datetime
import itertools
from datetime import date
#keys
consumer_key= "TMgCLhdfwOw5TcdqWyPNDqTAs"
consumer_secret= "ATYzvZerlvSd2SsInf9KLtgXAFILm3gXY6Vocj0S77MVM0VvBE"
access_token= "3297798173-zU5fbHhbYWAmGTCCX5loJITmgK19SHkrKxN9aF1"
access_token_secret= "xMYHAIe9D9YZGo8HaRHmIIoNeQg9QHmkENjUIeovdWKVS"

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# This requires Python’s OS module to create directory
import os.path

# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
crypto_tweets_document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="crypro_tweet_document_store")

#now we preprocess the textual data

from haystack.reader.farm import FARMReader
from haystack.utils import print_answers
from haystack.file_converter import TextConverter
from haystack.preprocessor.utils import convert_files_to_dicts
from haystack.preprocessor.preprocessor import PreProcessor
from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.pipeline import ExtractiveQAPipeline

#Sentiment Search Word prediction Imports
import nltk
from nltk import tokenize
from operator import itemgetter
import math
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')

#imports for Sentiment Analysis
import tweepy as tweepy
from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger
import nltk.sentiment
import nltk.sentiment.vader
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

#Creating PieCart
import matplotlib.pyplot as plt

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-nv7pe9xu
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-nv7pe9xu


/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
09/25/2021 10:30:07 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.007s]
09/25/2021 10:30:07 - INFO - elasticsearch -   HEAD http://localhost:9200/crypro_tweet_document_store [status:200 request:0.008s]
09/25/2021 10:30:07 - INFO - elasticsearch -   GET http://localhost:9200/crypro_tweet_document_store [status:200 request:0.006s]
09/25/2021 10:30:08 - INFO - elasticsearch -   PUT http://localhost:9200/crypro_tweet_document_store/_mapping [status:200 request:0.028s]
09/25/2021 10:30:08 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.004s]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### Hier wird festgelegt wohin die gesammelten Daten gespeichert werden, Außerdem kann man hier den Umfang der Suche mit den zu durchsuchenden Hastags festlegen

In [3]:
# 'mkdir' creates a directory in current directory.
if(os.path.isdir("/content/tweet_data/") == False):
  os.mkdir('tweet_data')

#define hashtags to search in
cryto_hashtags_string =  ["#btc","#eth","#ada","#crypto","#bnb","#usdt","#xrp","#usdc",
                          "#dot","#uni","#busd","#link","#doge","#sol","#bch","#ltc",
                          "#icp","#wbtc","#matic","#etc","#xlm","#theta","#vet","#luna",
                          "#fil","#trx","#dai","#aave","#xmr","#ftt","#eos","#axs",
                          "#cake","#grt","#cro","#btcb","#neo","#mkr","#klay","#atom",
                          "#shib","#avax","#xtz","#algo","#bsv","#miota","#btt","#leo",
                          "#egld","#comp","#amp"]
altcoins_hashtags_string = ["#btc","#eth","#ada","#crypto","#bnb",
                          "#dot","#uni","#link","#sol","#ltc",
                          "#wbtc","#matic",
                          "#cake"]

### Nun werden basierend auf den Hastags Daten über die Twitter API gefetched

In [4]:
#took 16 min for 51 hashtags
def flatten(t):
    return [item for sublist in t for item in sublist]
date_since = date.today()
print(date_since)
#inspired from https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists

all_user_locs = []
for entries in altcoins_hashtags_string:#cryto_hashtags_string:
  entries+="-filter:verified";
  tweets = tw.Cursor(api.search,
              q=entries, 
              lang="en",
              since=date_since).items(100)

  all_user_locs.append([[tweet.user.screen_name,tweet.user.created_at, tweet.text, tweet.favorite_count, tweet.entities["hashtags"]] for tweet in tweets])
users_locs = flatten(all_user_locs)

print(len(all_user_locs))
print(len(users_locs))

2021-09-25
13
1214


### Hier werden einige Preprocessing Schritte vorgenommen um die manchmal unübersichtlichen Tweets zu säubern. Diese werden dann auch anschließend gespeichert

In [5]:
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=["name","date", "content", "likes", "hashtags"])
def clean_data():

  #delete links
  tweet_text["content"].replace("((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*", "", regex = True, inplace = True)
                      
  #delete RT Tags
  tweet_text["content"].replace("RT", "", regex = True, inplace = True)

  print(len(tweet_text['content'].tolist()))
  #remove duplicates
  tweet_text.drop_duplicates(subset='content',keep='first', inplace=True)
  print(len(tweet_text['content'].tolist()))  

  #delete formatting
  #Quelle: https://gist.github.com/smram/d6ded3c9028272360eb65bcab564a18a
  tweet_text["content"].replace(to_replace=[r"\t|\n|\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

  #delete @names
  tweet_text["content"].replace("@([^\s]*)", "", regex = True, inplace = True)

  #delete hashtags
  #tweet_text["content"].replace("#([^\s]*)", "", regex = True, inplace = True)

  #delete leading whitespaces
  tweet_text["content"].replace(r"^ +| +$", r"", regex=True, inplace=True)

#adds a dot to the end of eachindividual tweet to make it readable as a sentence
def make_tweets_to_sentences(list):
  for tweets in list:
    tweets = tweets+"."
    print(tweets)
#removes all duplicates in the Tweets List
def remove_duplicates(my_list):
  temp_list = []
  for i in my_list:
    if i not in temp_list:
        temp_list.append(i)

  my_list = temp_list
  return my_list

clean_data()

content_list = tweet_text['content'].tolist()

#remove_duplicates(content_list)
# punkte werden hinzugefügt
make_tweets_to_sentences(content_list)
content_list

if((os.path.isfile("/content/tweet_data/tweet_data0.txt") == False)):
  print("doesnt exit")
  with open("tweet_data/tweet_data0.txt", "w") as output:
      output.write(str(content_list))
#if((os.path.isfile("/content/tweet_data/tweet_data0.txt") == True)):
#  print("does exit")
#  x=1;  
#  while (os.path.isfile("/content/tweet_data/tweet_data"+str(x)+".txt") == True):  
#    x=x+1;
#    with open("tweet_data/tweet_data"+str(x)+".txt", "w") as output:
#      output.write(str(content_list))
 


1214
664
#NanoDogeCoin is one of the chosen ones.                - Revolutionary refer….
#CryptoNews #binary #bitcoinexchange $dot #VitalikButerin #cryptocurrencymarket #SHIBASTRONG #xrpstandard….
If we burned 6 % with 45k holders in less the 3 months with 200k holders in one year how many #saitama will be burned? Th….
⚽#PremierLeague - In 1 hour! ⚽2 Games ⚽Bet Now!1️⃣Chelsea vs Manchester C ➡️2️⃣Aston Vi….
We are creating a decentralized and sustainable community $SUNI #SUNIMOON #BTC #ETH  #OHM.
#Bitcoin price in US Dollar #BTC $usd #BTCUSD $XBT #xbtusd BTC Price Now : 42,849.83$ USD Avg Last Hour : 42,785.….
#Bitgame #LUT #Bitgame_bounty #BTC #Crypto #Sports  promising project, it's nice to watch your….
📢 Pat Toomey, the top Republican on the Senate Banking Committee says #China’s crackdown on cryptocurrency transact….
Join #NanoDogeCoin- Revolutionary refer x (5% usdt per buy).- Amazing Passive income #usdt #ada #btc #eth….
#Cryptocurrencies Current Prices:#Bitcoin$ 42896.96€ 36672.

### Hier erstellen wir eine Datei, welche alle bereinigten Tweets umfasst, um das QA-System aufzubauen

In [6]:
#Now we created a file that contains all the saved tweets, we will build a QA system using Haystack in the following
all_collected_tweets = convert_files_to_dicts(dir_path="tweet_data/")

crypto_tweets_preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="sentence", # Dokument wird anhand von Sätzen getrennt
    split_length=10, # Nach 10 Sätzen erfolgt die Trennung -> ein Dokument besteht aus 10 Tweets
    split_overlap = 2, # 2 Sätze Überlappung bei den einzelnen Dokumenten
    split_respect_sentence_boundary=False
)
nested_docs = [crypto_tweets_preprocessor.process(doc) for doc in all_collected_tweets]
crypto_tweet_docs = [doc for x in nested_docs for doc in x]

print(f"n_files_input: {len(all_collected_tweets)}\nn_docs_output: {len(crypto_tweet_docs)}")

09/25/2021 10:31:33 - INFO - haystack.preprocessor.utils -   Converting tweet_data/tweet_data0.txt


n_files_input: 1
n_docs_output: 90


### Jetzt konfigurieren wir den Reader und Retriever unserer Suchmaschiene. Außerdem können wir hier festlegen welches Language Modell verwendet werden soll.

In [7]:
#Now we configure the reader and retriever
# "ahotrod/albert_xxlargev1_squad2_512"                     bis zu 22min
# "deepset/roberta-base-squad2"                             bis zu 2min
# "deepset/bert-large-uncased-whole-word-masking-squad2"    bis zu 5min
model_name_or_path = "deepset/bert-large-uncased-whole-word-masking-squad2"

crypto_tweets_document_store.write_documents(crypto_tweet_docs)
crypto_tweet_retriever = ElasticsearchRetriever(document_store=crypto_tweets_document_store)
crypto_tweets_reader = FARMReader(model_name_or_path, use_gpu=True,top_k=10)
crypto_pipe = ExtractiveQAPipeline(crypto_tweets_reader, crypto_tweet_retriever)

09/25/2021 10:31:34 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.150s]
09/25/2021 10:31:34 - INFO - farm.utils -   Using device: CPU 
09/25/2021 10:31:34 - INFO - farm.utils -   Number of GPUs: 0
09/25/2021 10:31:34 - INFO - farm.utils -   Distributed Training: False
09/25/2021 10:31:34 - INFO - farm.utils -   Automatic Mixed Precision: None
Some weights of the model checkpoint at deepset/bert-large-uncased-whole-word-masking-squad2 were not used when initializing BertModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeq

###Hier werden die Ergebnisse der Suchmaschiene angezeigt

In [ ]:
print(question_input_string)
#this step can take up to 20 min depending on what model is defined ("deepset/roberta-base-squad2" is quicker but doesnt retrieve quite good results)
crypto_prediction = crypto_pipe.run(query=question_input_string)
#, top_k_reader=5
print_answers(crypto_prediction, details="minimal")

09/25/2021 10:31:45 - INFO - elasticsearch -   POST http://localhost:9200/crypro_tweet_document_store/_search [status:200 request:0.076s]


Bitcoin performance


Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

### Hier werden aus der Suchanfrage mögliche Keywords extrahiert um einen Vorschag für die anstehende Sentiment Analyse zu generieren

In [ ]:
doc = question_input_string;
#https://www.analyticsvidhya.com/blog/2020/11/words-that-matter-a-simple-guide-to-keyword-extraction-in-python/
stop_words = set(stopwords.words('english'))
total_words = doc.split()
total_word_length = len(total_words)
#print(total_word_length)
total_sentences = tokenize.sent_tokenize(doc)
total_sent_len = len(total_sentences)
#print(total_sent_len)
tf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','')
    if each_word not in stop_words:
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

# Dividing by total_word_length for each dictionary element
tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
#print(len(tf_score.items()))
def retrieve_words(keyword_list):
  temp_string = "";
  for x in keyword_list:
    temp_string =temp_string +" "+ x
  #print(temp_string)
  return temp_string
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))
    
idf_score = {}
keyword_list= [];
for each_word in total_words:
    each_word = each_word.replace('.','')
    #print(each_word)
    if each_word not in stop_words:
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1
       
# Performing a log and divide
idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

#print(idf_score)
tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
#print(tf_idf_score)
def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    for x in result:
      x = x.replace('.','')
      keyword_list.append(x)
      #print(keyword_list)

    return result
get_top_n(tf_idf_score, 5)
retrieve_words(keyword_list) 

#print(tf_idf_score)


### Hier wird am Ende noch ein Sentiment Diagramm zu dem eingegebenen Suchbegriff erzeugt

In [ ]:

def percentage(part,whole):
    return 100 * float(part)/float(whole) 

suggestions = retrieve_words(keyword_list);
tweets_to_scan = 500;
keyword = input("Please enter keyword or hashtag to search: (Recommended: "+suggestions+" ) ")
noOfTweet = int(input ("Please enter how many tweets to analyze: (Recommended: "+str(tweets_to_scan)+" )"))

tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
positive  = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

for tweet in tweets:
    
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = sid.polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    
    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1

    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
    
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

#Number of Tweets (Total, Positive, Negative, Neutral)
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))



labels = ["Positive ["+str(positive)+"%]" , "Neutral ["+str(neutral)+"%]","Negative ["+str(negative)+"%]"]
sizes = [positive, neutral, negative]
colors = ["yellowgreen", "blue","red"]
patches, texts = plt.pie(sizes,colors=colors, startangle=90)
plt.style.use("default")
plt.legend(labels)
plt.title("Sentiment Analysis Result for keyword= "+keyword+"" )
plt.axis("equal")
plt.show()